In [1]:
import os

from tqdm import tqdm

import pandas as pd
import numpy as np

from run_kmeans import RunKmeans

from os.path import join, splitext
from numpy import min, mean, sum

In [2]:
pwd = os.getcwd()
datadir = join(pwd, 'data/')
outdir = join(pwd, 'out/')
files = os.listdir(datadir)
filename = 'boston.csv'
file = join(datadir, filename)
num_clusters = 25
outdir = join(outdir, str(num_clusters))
inits = [
    'random',
    'k-means++',
    'coc',
    'orss',
]

In [3]:
dfs = []
for file in tqdm(files[:]):
    filepath = join(datadir, file)
    comparator = RunKmeans(
        n_clusters=5,
        num_exp=20,
        file=filepath,
    )
    inertias, times, iters = comparator.run_kmeans()
#     print(file)
    rows = []
    for k in inertias:
        for inertia in inertias[k]:
            if type(inertia) == type(np.float64(1.0)):
                continue
            elif type(inertia) == type(np.int64(1.0)):
                continue
            else:
                raise TypeError(inertia, 'not float or int')
        row = []
        row.append(k)
#         print(k, mean(inertias[k]), mean(times[k]))
        row.append(round(mean(inertias[k]), 2))
        row.append(round(min(inertias[k]), 2))
        row.append(round(mean(times[k]), 2))
        row.append(round(min(times[k]), 2))
        row.append(round(mean(iters[k]), 2))
        row.append(round(min(iters[k]), 2))
        rows.append(row)
        columns=[
            'Initialization', 
            'Average Inertia', 
            'Minimum Inertia',
            'Average Time',
            'Minimum Time',
            'Average Iteration',
            'Minimum Iteration',
        ]
    df = pd.DataFrame(rows, columns=columns)
    dfs.append(df)

100%|████████████████████████████████████████████████████████████████████████████████████| 9/9 [02:31<00:00, 16.84s/it]


In [4]:
for i, df in enumerate(dfs):
    print(files[i])
    print(df.head())

airlines.csv
  Initialization  Average Inertia  Minimum Inertia  Average Time  \
0         random     5.788610e+12     5.788610e+12          1.59   
1      k-means++     5.803623e+12     5.724391e+12          1.45   
2           orss     5.898309e+12     5.724391e+12          0.95   
3            coc     5.800982e+12     5.724491e+12          1.41   

   Minimum Time  Average Iteration  Minimum Iteration  
0          1.36              34.10               29.0  
1          0.71              29.60               12.0  
2          0.57              18.05                9.0  
3          0.66              30.05               13.0  
boston.csv
  Initialization  Average Inertia  Minimum Inertia  Average Time  \
0         random       2612493.69       1476493.47          0.07   
1      k-means++       1676357.16       1475612.32          0.06   
2           orss       1649604.52       1475549.48          0.07   
3            coc       1940805.11       1476556.31          0.07   

   Minimum Tim

In [5]:
for i, df in enumerate(dfs):
    name, ext = splitext(files[i])
    file = join(outdir, name+'_out.csv')
    df.to_csv(file, index=False)